In [1]:
import pymysql
import json

coon = pymysql.connect(
    host = '10.15.82.58',user = 'root',passwd = '123',
    port = 3306,db = 'tcm',charset = 'utf8'
    )
source = coon.cursor()
source.execute("select json from qa_conclusion")
tem = source.fetchall()
source.close()
coon.close()

In [2]:
yufang = []
jiancha = []
yaowu = []
zhenjiu = []
tuina = []
huoguan = []
shoushu = []
pianfang = []
yangshen = []

buliangfanyin = []
paozhi = []
xiangguanjibin = []
yongfa = []
gongxiao = []

for single in tem:
    data = json.loads(single[0])
    for line in data['children']:
        if line['name'] == '预防':
            for sen in line['children']:
                yufang.append(sen['name'])
        elif line['name'] == '检查':
            for sen in line['children']:
                jiancha.append(sen['name'])
        elif line['name'] == '治疗':
            for nex in line['children']:
                if nex['name'] == '药物':
                    for nexx in nex['children']:
                        yaowu.append(nexx['name'])
                elif nex['name'] == '针灸':
                    for nexx in nex['children']:
                        zhenjiu.append(nexx['name'])
                elif nex['name'] == '推拿':
                    for nexx in nex['children']:
                        tuina.append(nexx['name'])
                elif nex['name'] == '火罐':
                    for nexx in nex['children']:
                        huoguan.append(nexx['name'])
                elif nex['name'] == '手术':
                    for nexx in nex['children']:
                        shoushu.append(nexx['name'])
                elif nex['name'] == '偏方':
                    for nexx in nex['children']:
                        pianfang.append(nexx['name'])
        elif line['name'] == '养生':
            for sen in line['children']:
                yangshen.append(sen['name'])
        elif line['name'] == '不良反应':
            for sen in line['children']:
                buliangfanyin.append(sen['name'])
        elif line['name'] == '炮制':
            for sen in line['children']:
                paozhi.append(sen['name'])
        elif line['name'] == '相关疾病':
            for sen in line['children']:
                xiangguanjibin.append(sen['name'])
        elif line['name'] == '用法':
            for sen in line['children']:
                yongfa.append(sen['name'])
        elif line['name'] == '功效':
            for sen in line['children']:
                gongxiao.append(sen['name'])

In [3]:
import sys
import os
import numpy as np
import random
from tqdm import tqdm
import jieba
import re
import keras
from keras import Input
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Dense, Flatten, Embedding, Dropout, concatenate, Bidirectional, LSTM, GRU, Reshape
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras_layer_normalization import LayerNormalization
from keras import backend as K
from keras.callbacks import Callback

def resample(data_dict,sample_size,seed,test_rate=0.1):
    """
    数据重采样
    :param data_dict: 输入数据dict类型：{label_1:[text_11,text_12,...,text_1j,...],label_2:[...],...,label_i:[...],...}
    :param sample_size: 每类样本容量：超过的样本类进行下采样，不足的样本类进行上采样
    :param seed: 随机种子
    :param test_rate: 测试集比例
    :return: train：训练集；test：测试集
    """
    random.seed(seed)
    np.random.seed(seed)
    train = []
    test = []
    for label in data_dict:
        #先抽取测试集
        num_data = len(data_dict[label])
        test_num_data = max(int(num_data // (1/test_rate)),1)
        for i in range(test_num_data):
            test_data = np.random.choice(data_dict[label],1)
            test_index = np.where(data_dict[label]==test_data)
            if test_num_data > 1:
                data_dict[label] = np.delete(data_dict[label],test_index)
            test.append([test_data[0],label])
        #再抽取训练集
        if len(data_dict[label]) >= sample_size:
            #下采样(不能重复，要用random.sample)
            train_data = random.sample(list(data_dict[label]),sample_size)
            train.extend([[train_data_i,label] for train_data_i in train_data])
        else:
            #上采样（有重复，要用np.random.choice+原数据）
            train_data = list(data_dict[label])
            train_data_extend = np.random.choice(train_data,sample_size-len(train_data)).tolist()
            train_data.extend(train_data_extend)
            train.extend([[train_data_i,label] for train_data_i in train_data])
    return train,test

regex = re.compile(u'[^\u4E00-\u9FA5|0-9a-zA-Z]')
def remove_punctuation(s):
    """
    文本标准化：仅保留中文字符、英文字符和数字字符
    :param s: 输入文本（中文文本要求进行分词处理）
    :return: s_：标准化的文本
    """
    s_ = regex.sub('', s)
    return s_

def text_tokenizer(texts):
    """
    文本分词+标准化
    :param texts: 输入文本list类型：[text_1,text_2,...,text_i,...]
    :return: 标准化后的分词文本
    """
    return [jieba.lcut(remove_punctuation(text)) for text in texts]

def BiGRU(train,test,val,output_categories,num_words,maxlen,embedding_dim,
          gru_units=256,dropout=0.2,recurrent_dropout=0.2,
          batch_size=512,epochs=10,verbose=1):
    """
    TextRNN模型：双向GRU
    :param train: 训练数据，格式：(data,label)
    :param test: 测试数据，格式：(data,label)
    :param val: 验证数据，格式：(data,label)
    :param output_categories: 分类类别数
    :param num_words: 限制top_n词数（文本内仅计算top_n的词向量）
    :param maxlen: 最大句长限制（计算基本单元：词组）
    :param embedding_dim: 词向量维度
    :param callbacks: 每个epoch进行回调计算的指标
    :param gru_units: GRU层神经元数量
    :param dropout:
    :param recurrent_dropout:
    :param batch_size:
    :param epochs:
    :param verbose:
    :return: model：模型；train_acc：训练精度；val_acc：验证精度；test_acc：测试精度
    """
    #一、数据准备
    train_data,train_label = train
    test_data,test_label = test
    val_data,val_label = val
    #一、网络构建
    inputs = Input(shape=(maxlen,))
    embedding = Embedding(input_dim=num_words,input_length=maxlen,output_dim=embedding_dim,trainable=True)(inputs)
    bigru = Bidirectional(GRU(units=gru_units,dropout=dropout,recurrent_dropout=recurrent_dropout,return_sequences=True))(embedding)
    bigru = Bidirectional(GRU(units=gru_units,dropout=dropout,recurrent_dropout=recurrent_dropout,activation='relu'))(bigru)
    outputs = Dense(output_categories,activation='softmax')(bigru)
    model = Model(inputs=inputs,outputs=outputs)
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-8),
              loss='categorical_crossentropy',
              metrics=['acc'])
    #二、模型训练及指标返回
    history = model.fit(train_data,train_label,batch_size=batch_size,validation_data=(val_data,val_label),epochs=epochs,verbose=verbose)
    train_acc = history.history['acc']
    val_acc = history.history['val_acc']
    test_loss,test_acc = model.evaluate(test_data,test_label)
    return model,train_acc,val_acc,test_acc


Using TensorFlow backend.
/data2/home/zju/workspace/bx/dead/anaconda3/envs/lsc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data2/home/zju/workspace/bx/dead/anaconda3/envs/lsc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data2/home/zju/workspace/bx/dead/anaconda3/envs/lsc/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [4]:
    # 1. 数据加载
    data=[]
    label=[]
#     data.extend(yufang)
#     data.extend(jiancha)
#     data.extend(yaowu)
#     data.extend(zhenjiu)
#     data.extend(tuina)
#     data.extend(huoguan)
#     data.extend(shoushu)
#     data.extend(pianfang)
#     data.extend(yangshen)
    data.extend(buliangfanyin)
    data.extend(paozhi)
    data.extend(xiangguanjibin)
    data.extend(yongfa)
    data.extend(gongxiao)
#     for i in range(len(yufang)):
#         label.append("预防")
#     for i in range(len(jiancha)):
#         label.append("检查")
#     for i in range(len(yaowu)):
#         label.append("药物")
#     for i in range(len(zhenjiu)):
#         label.append("针灸")
#     for i in range(len(tuina)):
#         label.append("推拿")
#     for i in range(len(huoguan)):
#         label.append("火罐")
#     for i in range(len(shoushu)):
#         label.append("手术")
#     for i in range(len(pianfang)):
#         label.append("偏方")
#     for i in range(len(yangshen)):
#         label.append("养生")
    for i in range(len(buliangfanyin)):
        label.append("不良反应")
    for i in range(len(paozhi)):
        label.append("炮制")
    for i in range(len(xiangguanjibin)):
        label.append("相关疾病")
    for i in range(len(yongfa)):
        label.append("用法")
    for i in range(len(gongxiao)):
        label.append("功效")

    data_dict = {}
    for data_i,label_i in zip(data,label):
        data_dict.setdefault(label_i,[]).append(data_i)

    sample_size = 1000
    seed = 0
    test_rate = 0.1
    train,test = resample(data_dict=data_dict,sample_size=sample_size,seed=seed,test_rate=test_rate)

    # 2. 数据处理
    tqdm_train = tqdm(train)
    tqdm_test = tqdm(test)
    tqdm_train.set_description("Train:")
    tqdm_train.set_description("Test:")
    train_label = []
    train_data = []
    test_label = []
    test_data = []
    try:
        for item in tqdm_train:
            data = item[0]
            label = item[1]
            train_label.append(label)
            train_data.append(text_tokenizer([data])[0])
    except KeyboardInterrupt:
        tqdm.close()
        raise
    try:
        for item in tqdm_test:
            data = item[0]
            label = item[1]
            test_label.append(label)
            test_data.append(text_tokenizer([data])[0])
    except KeyboardInterrupt:
        tqdm.close()
        raise

    label_set = []
    for label in train_label:
        if label not in label_set:
            label_set.append(label)

    label_convert2 = dict([[item,label_set.index(item)] for item in label_set])
    print(label_convert2)
    label_reconvert2 = dict([(label_convert2[key],key) for key in label_convert2])
    print(label_reconvert2)

    train_label = [label_convert2[item] for item in train_label]
    test_label = [label_convert2[item] for item in test_label]
    train_label = to_categorical(train_label)
    test_label = to_categorical(test_label)

    # 3. 文本序列化
    num_words = 10000
    maxlen = 25
    embedding_dim = 128
    output_categories = len(label_convert2)

    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(train_data)
    train_data = tokenizer.texts_to_sequences(train_data)
    test_data = tokenizer.texts_to_sequences(test_data)
    train_data = pad_sequences(train_data,maxlen=maxlen)
    test_data = pad_sequences(test_data,maxlen=maxlen)

    # 4. 输出指标
    target_names = [label_reconvert2[i] for i in range(len(label_reconvert2))]
    print(target_names)

Test::   0%|          | 0/5000 [00:00<?, ?it/s] Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.938 seconds.
Prefix dict has been built successfully.
Test:: 100%|██████████| 5000/5000 [00:01<00:00, 3422.99it/s]

100%|██████████| 1324/1324 [00:01<00:00, 831.51it/s]


{'不良反应': 0, '炮制': 1, '相关疾病': 2, '用法': 3, '功效': 4}
{0: '不良反应', 1: '炮制', 2: '相关疾病', 3: '用法', 4: '功效'}
['不良反应', '炮制', '相关疾病', '用法', '功效']


In [6]:
# 二、模型训练

    # 1. 设置基本超参数（未调参）

basic_hyper_parameters = {
        'train': (train_data, train_label),
        'test': (test_data, test_label),
        'val': (test_data, test_label),
        'output_categories': output_categories,
        'num_words': num_words,
        'maxlen': maxlen,
        'embedding_dim': embedding_dim
    }
bigruY, bigru_train_acc, bigru_val_acc, bigru_test_acc = BiGRU(**basic_hyper_parameters)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 25)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 128)           1280000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 512)           591360    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1181184   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 3,055,109
Trainable params: 3,055,109
Non-trainable params: 0
_________________________________________________________________

Train on 5000 samples, validate on 1324 samples
Epoch 1/10
5000/5000 [==============================] - 8s 2ms/ste

In [8]:
    # 1. 数据加载
    data=[]
    label=[]
    data.extend(yufang)
    data.extend(jiancha)
    data.extend(yaowu)
    data.extend(zhenjiu)
    data.extend(tuina)
    data.extend(huoguan)
    data.extend(shoushu)
    data.extend(pianfang)
    data.extend(yangshen)

    for i in range(len(yufang)):
        label.append("预防")
    for i in range(len(jiancha)):
        label.append("检查")
    for i in range(len(yaowu)):
        label.append("药物")
    for i in range(len(zhenjiu)):
        label.append("针灸")
    for i in range(len(tuina)):
        label.append("推拿")
    for i in range(len(huoguan)):
        label.append("火罐")
    for i in range(len(shoushu)):
        label.append("手术")
    for i in range(len(pianfang)):
        label.append("偏方")
    for i in range(len(yangshen)):
        label.append("养生")


    data_dict = {}
    for data_i,label_i in zip(data,label):
        data_dict.setdefault(label_i,[]).append(data_i)

    sample_size = 1000
    seed = 0
    test_rate = 0.1
    train,test = resample(data_dict=data_dict,sample_size=sample_size,seed=seed,test_rate=test_rate)

    # 2. 数据处理
    tqdm_train = tqdm(train)
    tqdm_test = tqdm(test)
    tqdm_train.set_description("Train:")
    tqdm_train.set_description("Test:")
    train_label = []
    train_data = []
    test_label = []
    test_data = []
    try:
        for item in tqdm_train:
            data = item[0]
            label = item[1]
            train_label.append(label)
            train_data.append(text_tokenizer([data])[0])
    except KeyboardInterrupt:
        tqdm.close()
        raise
    try:
        for item in tqdm_test:
            data = item[0]
            label = item[1]
            test_label.append(label)
            test_data.append(text_tokenizer([data])[0])
    except KeyboardInterrupt:
        tqdm.close()
        raise

    label_set = []
    for label in train_label:
        if label not in label_set:
            label_set.append(label)

    label_convert = dict([[item,label_set.index(item)] for item in label_set])
    print(label_convert)
    label_reconvert = dict([(label_convert[key],key) for key in label_convert])
    print(label_reconvert)

    train_label = [label_convert[item] for item in train_label]
    test_label = [label_convert[item] for item in test_label]
    train_label = to_categorical(train_label)
    test_label = to_categorical(test_label)

    # 3. 文本序列化
    num_words = 10000
    maxlen = 25
    embedding_dim = 128
    output_categories = len(label_convert)

    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(train_data)
    train_data = tokenizer.texts_to_sequences(train_data)
    test_data = tokenizer.texts_to_sequences(test_data)
    train_data = pad_sequences(train_data,maxlen=maxlen)
    test_data = pad_sequences(test_data,maxlen=maxlen)

    # 4. 输出指标
    target_names = [label_reconvert[i] for i in range(len(label_reconvert))]
    print(target_names)
    # 二、模型训练

    # 1. 设置基本超参数（未调参）

    basic_hyper_parameters = {
        'train': (train_data, train_label),
        'test': (test_data, test_label),
        'val': (test_data, test_label),
        'output_categories': output_categories,
        'num_words': num_words,
        'maxlen': maxlen,
        'embedding_dim': embedding_dim
    }
    bigruB, bigru_train_acc, bigru_val_acc, bigru_test_acc = BiGRU(**basic_hyper_parameters)

Test:: 100%|██████████| 9000/9000 [00:00<00:00, 10093.13it/s]

100%|██████████| 1778/1778 [00:01<00:00, 1642.51it/s]


{'预防': 0, '检查': 1, '药物': 2, '针灸': 3, '推拿': 4, '火罐': 5, '手术': 6, '偏方': 7, '养生': 8}
{0: '预防', 1: '检查', 2: '药物', 3: '针灸', 4: '推拿', 5: '火罐', 6: '手术', 7: '偏方', 8: '养生'}
['预防', '检查', '药物', '针灸', '推拿', '火罐', '手术', '偏方', '养生']
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 25, 128)           1280000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 25, 512)           591360    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               1181184   
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 4617      
Total params: 3,057,161
Trainable para

In [9]:
def predictB(new):
    new = text_tokenizer([new])
    new = tokenizer.texts_to_sequences(new)
    new = pad_sequences(new, maxlen=25)
    tem = bigruB.predict(new)
    return label_reconvert[tem.argmax()], tem.max()
predictB("我这个病需要动手术吗")
def predictY(new):
    new = text_tokenizer([new])
    new = tokenizer.texts_to_sequences(new)
    new = pad_sequences(new, maxlen=25)
    tem = bigruY.predict(new)
    return label_reconvert2[tem.argmax()], tem.max()
predictY("这个药吃下去有效果吗")

('炮制', 0.48746264)

In [10]:
predictB("我这个病需要动手术吗")

('手术', 0.8269756)

In [11]:
label_reconvert

{0: '预防',
 1: '检查',
 2: '药物',
 3: '针灸',
 4: '推拿',
 5: '火罐',
 6: '手术',
 7: '偏方',
 8: '养生'}

In [12]:
entities = set()
import pymysql
coon = pymysql.connect(
    host = '10.15.82.50',user = 'root',passwd = '123',
    port = 3306,db = 'lsc',charset = 'utf8'
    )
data = coon.cursor()
data.execute("select entity,type from conc group by entity having count(*)>10")
tem = data.fetchall()
for line in tem:
    entities.add(line)

In [13]:
coon2 = pymysql.connect(
    host = '10.15.82.58',user = 'root',passwd = '123',
    port = 3306,db = 'qa',charset = 'utf8'
    )
q2 = 'select qa_q from qa_new where qa_id = %s'
cur2 = coon2.cursor()

In [ ]:
q1 = "select id from conc where entity = %s"
for entityT in entities:
    entity = entityT[0]
    BY = entityT[1]
    data.execute(q1,(entity))
    for line in data.fetchall():
        cur2.execute(q2,(line[0]))
        sent = cur2.fetchall()[0][0][:150]
        if BY == "drug":
            if "鉴别" in sent or "假冒" in sent or "识别" in sent or "区分" in sent or "混淆" in sent:
                pred = ("鉴别", 1)
            else:
                pred = predictY(sent)
        else:
            pred = predictB(sent)
        if pred[1] >= 0.40:
            query = 'insert into conc3(entity, question, type, id) values(%s, %s, %s, %s)'
            data.execute(query,(entity, sent, pred[0], line[0]))
            coon.commit()